In [1]:
import networkx as nx
import torch
from torch_geometric.utils.convert import from_networkx

In [2]:
#Part 1

#Task 1.1
def gen_cycle_pairs(ns):
    graphs = []
    
    #generate pairs between 3 and n-3 such that their sum equals n
    def gen_pairs(n):
        res = []
        for i in range(3, n-2):
            res.append((i, n-i))
        return res
    
    #iterate over all ns 
    for n in ns:
        
        #generate pairs for given n and iterate over all pairs
        cycles = gen_pairs(n)
        for cycle in cycles:
            #for each pair generate the corresponing graphs
            c1 = nx.cycle_graph(cycle[0])
            c2 = nx.cycle_graph(cycle[1])
            graphs.append(nx.disjoint_union(c1, c2))
            
            #also append a single cycle graph for balance
            graphs.append(nx.cycle_graph(n))  
    
    return graphs
    
nx_graphs = gen_cycle_pairs(range(6, 16))

#Sanity check
for i in range(0, 110,2):  
    assert(nx.algorithms.graph_hashing.weisfeiler_lehman_graph_hash(nx_graphs[i]) 
        == nx.algorithms.graph_hashing.weisfeiler_lehman_graph_hash(nx_graphs[i+1]))

    
#Task 1.2
py_graphs = []
for i, graph in enumerate(nx_graphs):

    py_graph = from_networkx(graph)
    
    #x features
    py_graph.x = torch.zeros(py_graph.num_nodes,50)
    
    #for all even indices the graph is not a simple cycle
    if (i%2 == 0):
        py_graph.y = torch.tensor([0])
        
    #for all odd indices the graph is a simple cycle
    else:
        py_graph.y = torch.tensor([1])
        
    py_graphs.append(py_graph)
    


In [ ]:
#Part 2
import torch.nn as nn
import torch_geometric.nn as tg_nn
import torch.nn.functional as F
from torch_geometric.data import DataLoader

#Task 2.1

input_dim = 50
output_dim = 1

class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        #ModuleList of 16 Message-passing layers
        self.convs = nn.ModuleList()
        self.convs.append(tg_nn.GCNConv(input_dim, 50))
        
        self.num_MP_layers = 16
        
        for i in range(15):
            self.convs.append(tg_nn.GCNConv(50, 50))
        
        #MLPs for post processing
        self.MLPs = nn.Sequential(
            nn.Linear(50, 50), nn.Dropout(0.25),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50, output_dim)
            )
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        #forwards throug MPNNs
        for i in range(self.num_MP_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, 0.25)
        
        #global mean pool
        x = tg_nn.global_mean_pool(x, batch)
        
        #MLPs
        x = self.MLPs(x)     
        return x
    
    #reset parameters of every parameterised layer
    def reset_parameters(self):
        for layers in self.children():
            for layer in layers:
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
    
    #one output feature -> BCE loss
    def loss(self, pred, label):       
        return F.binary_cross_entropy(F.sigmoid(pred), label.float().unsqueeze(1))
    
#Task 2.2

def train(net, training_loader, epochs):
    
    #Adam with 1.e-4 learning rate
    opt = torch.optim.Adam(net.parameters(), lr = 1.e-4)

    for epoch in range(1,epochs+1):
        total_loss = 0
        net.train()
        for batch in training_loader:
            opt.zero_grad()
            
            prediction = net(batch)          
            label = batch.y
            
            loss = net.loss(prediction, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
            
        total_loss /= len(training_loader.dataset)
        
        print(f'Epoch {epoch+0:03}: | Loss: {total_loss:.5f}')

def test(net,loader):
    
    net.eval()

    correct = 0

    for data in loader:
        with torch.no_grad():
            output = net(data)
            
            #sigmoid(output) > 0.5 -> true, else false
            classification = torch.round(F.sigmoid(output))
            label = data.y
            #print('classification', classification, 'label', label)
        
        correct+= (classification == label).sum()

    return correct/ len(loader.dataset)

def cross_validation(net, dataset, epochs):
    data_size = len(dataset)
    fold_index = int(0.2*len(dataset))
    
    mean_accuracy = 0
    for k in range(5):
        
        #reset parameters after starting new fold
        net.reset_parameters()
        
        #use 6 indices for left and right bound of the three data chunks
        #[train_index_left_left, train_index_left_right],[test_index_left, test_index_right],
        #[train_index_right_left, train_index_right_right]
        
        train_index_left_left = 0
        train_index_left_right = k * fold_index

        test_index_left = train_index_left_right
        test_index_right = test_index_left + fold_index

        train_index_right_left = test_index_right
        train_index_right_right = data_size

        #training set is union of two training chunks
        train_indices = list(range(train_index_left_left, train_index_left_right))+ list(range(train_index_right_left, train_index_right_right))
        test_indices = list(range(test_index_left, test_index_right))

        train_set = torch.utils.data.dataset.Subset(dataset,train_indices)
        test_set = torch.utils.data.dataset.Subset(dataset, test_indices)

        train_loader = DataLoader(train_set, shuffle = True)
        test_loader = DataLoader(test_set, shuffle = True)

        train(net, train_loader, epochs)
        
        accuracy = test(net, test_loader)
        mean_accuracy+= accuracy
        print(f'\nFold {k}: | Accuracy: {accuracy:.5f}\n')
        
    print('Total Mean Accuracy', mean_accuracy/5)


net = Network()
net = cross_validation(net, py_graphs, 200)



/Users/jonaswildberger/Library/Python/3.8/lib/python/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 001: | Loss: 0.69380
Epoch 002: | Loss: 0.69354
Epoch 003: | Loss: 0.69482
Epoch 004: | Loss: 0.69631
Epoch 005: | Loss: 0.69285
Epoch 006: | Loss: 0.69359
Epoch 007: | Loss: 0.69486
Epoch 008: | Loss: 0.69558
Epoch 009: | Loss: 0.69373
Epoch 010: | Loss: 0.69332
Epoch 011: | Loss: 0.69449
Epoch 012: | Loss: 0.69541
Epoch 013: | Loss: 0.69394
Epoch 014: | Loss: 0.69426
Epoch 015: | Loss: 0.69394
Epoch 016: | Loss: 0.69344
Epoch 017: | Loss: 0.69275
Epoch 018: | Loss: 0.69223
Epoch 019: | Loss: 0.69603
Epoch 020: | Loss: 0.69357
Epoch 021: | Loss: 0.69587
Epoch 022: | Loss: 0.69303
Epoch 023: | Loss: 0.69240
Epoch 024: | Loss: 0.69588
Epoch 025: | Loss: 0.69094
Epoch 026: | Loss: 0.69583
Epoch 027: | Loss: 0.69511
Epoch 028: | Loss: 0.69700
Epoch 029: | Loss: 0.69570
Epoch 030: | Loss: 0.69378
Epoch 031: | Loss: 0.69419
Epoch 032: | Loss: 0.69232
Epoch 033: | Loss: 0.69555
Epoch 034: | Loss: 0.69371
Epoch 035: | Loss: 0.69185
Epoch 036: | Loss: 0.69083
Epoch 037: | Loss: 0.69470
E

Epoch 104: | Loss: 0.69450
Epoch 105: | Loss: 0.69463
Epoch 106: | Loss: 0.69530
Epoch 107: | Loss: 0.69292
Epoch 108: | Loss: 0.69320
Epoch 109: | Loss: 0.69348


In [ ]:
#Part 3

#Tast 3.1
class Network_RNI(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.convs = nn.ModuleList()
        self.convs.append(tg_nn.GCNConv(input_dim, 50)
        )
        
        self.num_MP_layers = 16
        
        for i in range(15):
            self.convs.append(tg_nn.GCNConv(50, 50))
        
        self.MLPs = nn.Sequential(
            nn.Linear(50, 50), nn.Dropout(0.25),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50, output_dim)
            )
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        #sample 25 random features for ever data point
        rvec = torch.randn((len(batch),25))
        
        x_pre = x.narrow(1, 0,25)
        
        #concatenate 25 zeros with 25 N(0,1) values for consecutive processing
        x = torch.cat((x_pre, rvec), dim = 1)
        #print(x)
        
        for i in range(0,self.num_MP_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, 0.25)
            
        x = tg_nn.global_max_pool(x, batch)
        
        x = self.MLPs(x)
        
        
        return x
    
    def reset_parameters(self):
        for layers in self.children():
            for layer in layers:
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
    
    def loss(self, pred, label):
        return F.binary_cross_entropy(F.sigmoid(pred), label.float().unsqueeze(1))

net_rni = Network_RNI()
print(net_rni)

cross_validation(net_rni, py_graphs, 400)

In [ ]:
class Network_RNI_adapted(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.convs = nn.ModuleList()
        self.convs.append(tg_nn.GCNConv(input_dim, 50)
        )
        #4 MP layers
        self.num_MP_layers = 4
        
        for i in range(3):
            self.convs.append(tg_nn.GCNConv(50, 50))
        
        #8 MLP layers
        self.MLPs = nn.Sequential(
            nn.Linear(50, 50), nn.Dropout(0.25),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50,50), nn.Dropout(0.1),
            nn.Linear(50, output_dim)
            )
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        rvec = torch.randn((len(batch),25))
        
        x_pre = x.narrow(1, 0,25)
        
        x = torch.cat((x_pre, rvec), dim = 1)
        #print(x)
        
        for i in range(0,self.num_MP_layers):
            #print(x.shape)
            #print(x)
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, 0.25)
            
        x = tg_nn.global_max_pool(x, batch)
        
        x = self.MLPs(x)
        
        
        return x
    
    def reset_parameters(self):
        for layers in self.children():
            for layer in layers:
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
    
    def loss(self, pred, label):
        #print('lossfuntion', F.sigmoid(pred[0]))
        return F.binary_cross_entropy(F.sigmoid(pred), label.float().unsqueeze(1))
    
    
    
net_rni_adapted = Network_RNI_adapted()

cross_validation(net_rni_adapted, py_graphs, 400)